In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time
from selenium.webdriver.common.by import By 


Inititialization

In [2]:

def load_data(url, driver_path):
    SCROLL_DOWN = "window.scrollTo(0, document.body.scrollHeight);"
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(5)
    
    load_button = driver.find_element(By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")
    
    for i in range(0, 3):
        print("Scrolling down...")
        driver.execute_script(SCROLL_DOWN)
        time.sleep(2)
        load_button.click()
        time.sleep(10)
        
    ret = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
        
    return ret

def get_review(TITLE_CODE):
    review_url = f'https://www.imdb.com/title/{TITLE_CODE}/reviews?spoiler=hide&sort=reviewVolume&dir=desc&ratingFilter=0'
    soup = BeautifulSoup(requests.get(review_url).content, 'html.parser')
    review = soup.find_all('a', class_='title')
    return [r.text for r in review]

def get_data_for(
        HTML_CONTENT,
        imdb_movie_title_code,
        movie_names,
        movie_release_date,
        movie_duration_time,
        movie_ratings,
        movie_metascore,
        reviews
    ):
        
    href = HTML_CONTENT.find_all(
        'div',
        class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-479faa3c-9 dkLVoC dli-title"
    )
    
    names = HTML_CONTENT.find_all(
        'h3',
        class_='ipc-title__text'
    )
    
    year = HTML_CONTENT.find_all(
        'span',
        class_='sc-479faa3c-8 bNrEFi dli-title-metadata-item'
    )[0]
    
    time = HTML_CONTENT.find_all(
        'span',
        class_='sc-479faa3c-8 bNrEFi dli-title-metadata-item'
    )[1]
    
    rating = HTML_CONTENT.find_all(
        'svg',
        class_='ipc-icon ipc-icon--star-inline'
    )
    
    meta_rating = HTML_CONTENT.find_all(
        'span',
        class_='sc-b0901df4-0 bcQdDJ metacritic-score-box'
    )
    
    for href in href:
        imdb_movie_title_code.append(href.find('a')['href'].split('/')[2])
    
    for mov in names:
        movie_names.append(mov.text)
    
    for y in year:
        movie_release_date.append(y.text)
    
    for t in time:
        movie_duration_time.append(t.text)
        
    for r in rating:
        movie_ratings.append(r.text)
    
    for m in meta_rating:
        movie_metascore.append(m.text)
    
    for i, j in zip(imdb_movie_title_code, range(0, len(imdb_movie_title_code))):
        reviews.append(get_review(i))
        print(f"Done for {j + 1}: ({i})")
    


In [ ]:
DRIVER_PATH = "A:\chromedriver-win64\chromedriver.exe"
URL_MOVIES = 'https://www.imdb.com/search/title/?sort=user_rating,asc&groups=top_1000&count=250'
URL_TVSHOWS = 'https://www.imdb.com/search/title/?title_type=tv_series&sort=num_votes,desc&num_votes=1000,&languages=en&count=250'

HTML_CONTENT_MOV = load_data(URL_MOVIES, DRIVER_PATH)
# HTML_CONTENT_TV = load_data(URL_TVSHOWS, DRIVER_PATH)


In [ ]:
# temporary test each segment
movie_release_date = []
year = HTML_CONTENT_MOV.find_all(
    "span", class_="sc-479faa3c-8 bNrEFi dli-title-metadata-item"
)
for y in year:
    print(y)

print(movie_release_date)

In [ ]:
imdb_movie_title_code = []
movie_names = []
movie_reviews = []
movie_ratings = []
# movie_directors = []
# movie_stars = []
movie_release_date = []
movie_metascore = []
movie_duration_time = []
# movie_genres = []




get_data_for(
    HTML_CONTENT_MOV,
    imdb_movie_title_code,
    movie_names,
    movie_release_date,
    movie_duration_time,
    movie_ratings,
    movie_metascore,
    movie_reviews
)




dataframe_mov = {
    'Name': movie_names[:-1],
    'Release Date': movie_release_date,
    'Duration': movie_duration_time,
    'Rating': movie_ratings,
    'Metascore': movie_metascore,
    'title_code': imdb_title_code,
    'reviews': reviews,
}


pd.DataFrame(dataframe_mov).to_excel('movies.xlsx', index=False)
